<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Alternate-Commits/blob/master/09_10_22_politifact_pytorch_biggraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -qq install jsonlines

In [2]:
!git clone -qb working https://github.com/arampacha/PyTorch-BigGraph

In [3]:
%cd PyTorch-BigGraph/
!pip -qq install .
%cd /content/

/content/PyTorch-BigGraph
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
/content


In [4]:
from google.colab import drive
drive.mount('/gdrive')

from pathlib import Path
base_dir = Path("/gdrive/MyDrive/ResearchFND")
assert base_dir.exists()

Mounted at /gdrive


## Data prep

In [5]:
import pandas as pd
import ast

In [6]:
df = pd.read_csv(base_dir/'politifact_agg.csv', index_col=0)
df.head(2)

,title,text,tweets,retweets,label,url,tweet_ids,num_retweets,log_num_retweets,num_tweets,log_num_tweets
0,Actress Emma Stone ‘For the first time in his...,NaN,[],"['1020554564334964741', '1020817527046197248',...",fake,NaN,[],2911,7.976595,0,0.0
1,Breaking President Trump makes English the of...,NaN,[],[],fake,NaN,[],0,0.000000,0,0.0


In [7]:
df['tweets'] = df.tweets.map(ast.literal_eval)

In [8]:
users_tweeted = df.tweets.map(lambda x: [int(e['user_id']) for e in x])

In [9]:
len(users_tweeted), sum(users_tweeted.map(len) > 0)

(894, 149)

In [10]:
import os
import jsonlines
import numpy as np
import torch

import IPython.display as ipd

In [11]:
follow_src = []
follow_dst = []
with jsonlines.open(base_dir/"followers.jsonl") as reader:
    for line in reader:
        v = line["user_id"]
        for u in line["followers"]:
            follow_src.append(u)
            follow_dst.append(v)

In [12]:
with jsonlines.open(base_dir/"following.jsonl") as reader:
    for line in reader:
        u = line["user_id"]
        for v in line["following"]:
            follow_src.append(u)
            follow_dst.append(v)

In [13]:
tweet_src = []
tweet_dst = []

for v, l in users_tweeted.iteritems():
    if not len(l):
        continue
    for u in l:
        tweet_src.append(u)
        tweet_dst.append(v)

In [14]:
with open('edges.txt', 'w') as f:
    for src, dst in zip(follow_src, follow_dst):
        f.write(f"{src}\t{dst}\tfollows\n")
    for src, dst in zip(tweet_src, tweet_dst):
        f.write(f"{src}\t{dst}\ttwitted\n")

In [15]:
!head -n 5 edges.txt

983553159057498114	961251714857828357	follows
988529911873921024	961251714857828357	follows
961635897929359360	961251714857828357	follows
159717173	961251714857828357	follows
4737344780	961251714857828357	follows


In [16]:
!cp edges.txt $base_dir/

## Training

In [17]:
import random
from pathlib import Path

import attr
import pkg_resources
from torchbiggraph.config import add_to_sys_path, ConfigFileLoader
from torchbiggraph.converters.importers import convert_input_data, TSVEdgelistReader
from torchbiggraph.converters.utils import download_url, extract_gzip, extract_tar
from torchbiggraph.eval import do_eval
from torchbiggraph.train import train
from torchbiggraph.util import (
    set_logging_verbosity,
    setup_logging,
    SubprocessInitializer,
)

In [18]:
data_dir = Path('./data')
data_dir.mkdir(parents=True, exist_ok=True)
fpath = base_dir/'edges.txt'

In [19]:
import random

def split_edges(input_file, pct=0.8, train_file='data/train_edges.txt', valid_file='data/valid_edges.txt'):

    with open(input_file) as f:
        lines = f.readlines()

    follow_edges, tweet_edges = [], []
    for line in lines:
        if line.strip().endswith('follows'):
            follow_edges.append(line)
        else:
            tweet_edges.append(line)

    random.shuffle(follow_edges)
    random.shuffle(tweet_edges)
    follow_split, tweet_split = int(pct*len(follow_edges)), int(pct*len(tweet_edges))
    train_edges = follow_edges[:follow_split] + tweet_edges[:tweet_split]
    valid_edges = follow_edges[follow_split:] + tweet_edges[tweet_split:]

    with open(train_file, 'w') as f:
        f.writelines(train_edges)

    with open(valid_file, 'w') as f:
        f.writelines(valid_edges)

In [20]:
split_edges(fpath)

In [21]:
loader = ConfigFileLoader()
config = loader.load_config('PyTorch-BigGraph/torchbiggraph/examples/configs/politifact_config.py', [])
set_logging_verbosity(0)
subprocess_init = SubprocessInitializer()
subprocess_init.register(setup_logging, 0)
subprocess_init.register(add_to_sys_path, loader.config_dir.name)
input_edge_paths = [data_dir/'train_edges.txt', data_dir/'valid_edges.txt']
output_train_path, output_test_path = config.edge_paths

In [22]:
convert_input_data(
    config.entities,
    config.relations,
    config.entity_path,
    config.edge_paths,
    input_edge_paths,
    TSVEdgelistReader(lhs_col=0, rhs_col=1, rel_col=2),
    dynamic_relations=config.dynamic_relations,
)

[2022-09-10 11:50:31.895850] Using the 2 relation types given in the config
[2022-09-10 11:50:31.902427] Searching for the entities in the edge files...
[2022-09-10 11:50:32.408878] Entity type user:
[2022-09-10 11:50:32.453695] - Found 71759 entities
[2022-09-10 11:50:32.459189] - Removing the ones with fewer than 1 occurrences...
[2022-09-10 11:50:32.527051] - Left with 71759 entities
[2022-09-10 11:50:32.535018] - Shuffling them...
[2022-09-10 11:50:32.673179] Entity type article:
[2022-09-10 11:50:32.680362] - Found 149 entities
[2022-09-10 11:50:32.685822] - Removing the ones with fewer than 1 occurrences...
[2022-09-10 11:50:32.691534] - Left with 149 entities
[2022-09-10 11:50:32.697291] - Shuffling them...
[2022-09-10 11:50:32.699117] Preparing counts and dictionaries for entities and relation types:
[2022-09-10 11:50:32.703313] - Writing count of entity type user and partition 0
[2022-09-10 11:50:32.807161] - Writing count of entity type article and partition 0
[2022-09-10 11:

In [23]:
train_config = attr.evolve(config, edge_paths=[output_train_path])
train(train_config, subprocess_init=subprocess_init)

INFO:torchbiggraph:Loading entity counts...
INFO:torchbiggraph:Creating workers...
INFO:torchbiggraph:Initializing global model...
INFO:torchbiggraph:Starting epoch 1 / 30, edge path 1 / 1, edge chunk 1 / 1
INFO:torchbiggraph:Edge path: data/train_partitioned
INFO:torchbiggraph:still in queue: 0
INFO:torchbiggraph:Swapping partitioned embeddings None ( 0 , 0 )
INFO:torchbiggraph:Loading partitioned embeddings from checkpoint
INFO:torchbiggraph:( 0 , 0 ): Stats before training: loss:  465.565 , pos_rank:  958.222 , mrr:  0.00351497 , r1:  0.000336776 , r10:  0.00336776 , r50:  0.0242479 , auc:  0.489897 , count:  4454
INFO:torchbiggraph:( 0 , 0 ): Training stats: loss:  20.4204 , reg:  0 , violators_lhs:  37.6978 , violators_rhs:  37.7265 , count:  84644
INFO:torchbiggraph:( 0 , 0 ): Stats after training: loss:  320.296 , pos_rank:  564.79 , mrr:  0.0854129 , r1:  0.06859 , r10:  0.111024 , r50:  0.180512 , auc:  0.71621 , count:  4454
INFO:torchbiggraph:( 0 , 0 ): bucket 1 / 1 : Traine

In [24]:
eval_config = attr.evolve(config, edge_paths=[output_test_path])
do_eval(eval_config, subprocess_init=subprocess_init)

INFO:torchbiggraph:Starting edge path 1 / 1 (data/test_partitioned)
INFO:torchbiggraph:( 0 , 0 ): Processed 22275 edges in 1.5 s (0.015M/sec); load time: 0.079 s
INFO:torchbiggraph:Stats for edge path 1 / 1, bucket ( 0 , 0 ): loss:  17.4526 , pos_rank:  31.4775 , mrr:  0.185657 , r1:  0.106846 , r10:  0.358182 , r50:  0.735623 , auc:  0.705073 , count:  22275
INFO:torchbiggraph:
INFO:torchbiggraph:Stats for edge path 1 / 1: loss:  17.4526 , pos_rank:  31.4775 , mrr:  0.185657 , r1:  0.106846 , r10:  0.358182 , r50:  0.735623 , auc:  0.705073 , count:  22275
INFO:torchbiggraph:
INFO:torchbiggraph:
INFO:torchbiggraph:Stats: loss:  17.4526 , pos_rank:  31.4775 , mrr:  0.185657 , r1:  0.106846 , r10:  0.358182 , r50:  0.735623 , auc:  0.705073 , count:  22275
INFO:torchbiggraph:


In [25]:
!torchbiggraph_export_to_tsv \
    'PyTorch-BigGraph/torchbiggraph/examples/configs/politifact_config.py' \
    --entities-output entity_embeddings.tsv \
    --relation-types-output relation_types_parameters.tsv

Loading relation types and entities...
Initializing model...
Loading model check point...
Writing entity embeddings...
Reading embeddings for entity type user partition 0 from checkpoint...
Writing embeddings for entity type user partition 0 to output file...
- Processed 5000/71759 entities so far...
- Processed 10000/71759 entities so far...
- Processed 15000/71759 entities so far...
- Processed 20000/71759 entities so far...
- Processed 25000/71759 entities so far...
- Processed 30000/71759 entities so far...
- Processed 35000/71759 entities so far...
- Processed 40000/71759 entities so far...
- Processed 45000/71759 entities so far...
- Processed 50000/71759 entities so far...
- Processed 55000/71759 entities so far...
- Processed 60000/71759 entities so far...
- Processed 65000/71759 entities so far...
- Processed 70000/71759 entities so far...
- Processed all 71759 entities
Reading embeddings for entity type article partition 0 from checkpoint...
Writing embeddings for entity type